In [1]:
%run setup_Sv_compute.ipynb

# of files w/o bandwidth: 57
NB-VM-150         1088
OS-38              519
OS-75              363
WH-300             188
OS-150              26
NB-VM-300           25
NB-VM-75            14
BB-VM-150           13
OS-II-38             6
WH-Mariner-300       3
NB-DR-150            3
WH-1200              1
OS-II-75             1
DCP4400A             1
WH-Mariner-600       1
Name: instrument_name, dtype: int64


In [2]:
%run fxns_Sv_compute.ipynb

# Compute Sv for a few files w/ diff characteristics

In [ ]:
pd.options.display.max_rows = 999
dfnow = df[df['geo_region'].notnull()]
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
#dfnow[['fname','geo_region','instrument_name']]

In [ ]:
#fnamenow = '00890_short.nc' # --> Sv computation works for this! Uses OS, in central eq pac
fnamenow = '00111_short.nc' # --> Sv computation works for this! Uses NB-VM-150, in w eq pac

In [ ]:
ncfile = dpath + 'JASADCP/ncfiles/' + fnamenow
nc = xr.open_dataset(ncfile)
#strnow = nc.attrs['cruise_sonar_summary']
#re.findall('arro', strnow, re.IGNORECASE)
#nc.attrs['cruise_sonar_summary'].split('\n')
#nc
#print(nc['depth'])
#nc['amp']
#nc['amp'].plot()
#nc['depth'].values
#nc['ADCP_CONFIG'].attrs['model_name']
#nc['ADCP_CONFIG'].attrs

In [ ]:
amp = nc['amp']
Tx = nc['tr_temp']
depth = nc['depth']

In [ ]:
row = df[df['fname']==fnamenow].iloc[0]

In [ ]:
# - Testing each fxn
Er = get_Er_constant(row,amp)
print('Er: ',Er)

if 'NB' not in row['instrument_name']:
    Kc = get_Kc_constant(row)
elif 'NB' in row['instrument_name']:
    Kc = get_Kc_tdresolved(row,amp,Tx)
print('Kc: ',Kc)

LDBM = get_LDBM_constant(row)
print('LDBM: ',LDBM)

if 'NB' not in row['instrument_name']:
    PDBW = get_PDBW_constant(row)
elif 'NB' in row['instrument_name']:
    #PDBW = get_PDBW_tdresolved(row)
    PDBW = get_PDBW_constant(row)
print('PDBW: ',PDBW)

R = get_R_tdresolved(row, depth, method_num=5)
print('R: ',R)

# - no bandwidth for a file; pretend it's narrowband for now
#row['bandwidth']='narrowband'

c = calc_c_tdresolved(depth,T=25,S=35)
print('c: ',c)

if 'NB' not in row['instrument_name']:
    C = get_C_constant(row)
elif 'NB' in row['instrument_name']:
    C = get_C_tdresolved(row,c)
print('C: ',C)

Tnow = depth; Tnow.name = 'temperature'; Tnow[:,:] = 25
alpha = calc_alpha_tdresolved(row,depth,c,Tnow,S=35,pH=8.1)
print('alpha: ',alpha)

alphaR = calc_alphaR_tdresolved(alpha,R)
print('alphaR: ',alphaR)

In [ ]:
Sv = C + 10*np.log10((Tx+273.16)*R**2) - LDBM + PDBW \
    + 2*alphaR + 10*np.log10(10**(Kc*(amp-Er)/10) - 1)

In [ ]:
amp.plot(y='depth_cell')
plt.gca().invert_yaxis()

In [ ]:
Sv.plot(y='depth_cell', vmax=0)
plt.gca().invert_yaxis()

# OLD NOTES

In [ ]:
fnames = sorted(os.listdir(dpath + 'JASADCP/ncfiles'))

In [ ]:
nc_counter = len(fnames)
geo_region = [None]*nc_counter

In [ ]:
ifile = 0
for fname in fnames:
    ncfile = dpath + 'JASADCP/ncfiles/' + fname
    ncnow = xr.open_dataset(ncfile)
    strnow = ncnow.attrs['cruise_sonar_summary']
    geo_regionnow = re.findall("GEOGRAPHIC_REGION *: *((?:\S+ )*\S+)", strnow)
    if geo_regionnow:
        geo_region[ifile] = geo_regionnow[0]
    ifile = ifile+1

In [ ]:
df = pd.concat(
    [pd.Series(fnames,name='fname'), pd.Series(geo_region,name='geo_region')],
    axis=1)

In [ ]:
print(df['geo_region'].isnull().sum())
print(len(df))
dfnow = df.dropna(subset=['geo_region'])
print(len(dfnow))
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
print(len(dfnow))
dfnow = dfnow[dfnow['instrument_name'].str.contains('OS')]
print(len(dfnow))
dfnow = dfnow[(dfnow['instrument_name']!='OS-II-38')&(dfnow['instrument_name']!='OS-II-75')] 

## Load table of kc, Er values

#### Nominal Kc values from TRDI field service email:
BBADCP: 0.45 dB/count  
WHADCP: 0.42 dB/count  
OS-II-75, SN1508: 0.373 (beam 1), 0.386 (beam 2), 0.388 (beam 3), 0.384 (beam 4) = 0.38275 (avg)  
OS-II-75, SN10656: 0.398 (beam 1), 0.389 (beam 2), 0.3988 (beam 3), 0.395 (beam 4) = 0.395 (avg)  
#### Nominal Kc values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 0.41-0.44, 0.42, 0.012   
OS75: 0.36-0.42, 0.39, 0.019  
OS38: 0.36-0.37, 0.37, 0.004  

#### Nominal Er values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 18-28, 22, 2.08  
OS75: 11-25, 19, 2.93  
OS38: 5-33, 14, 4.83  

In [ ]:
# - Calc avg Kc for OS-II w/ SNs
print(np.mean([0.373, 0.386, 0.388, 0.384]))
print(np.mean([0.398, 0.389, 0.398, 0.395]))